In [1]:
from feon.sa import *
from feon.tools import pair_wise
import numpy as np 
import pandas as pd
import feon as  fn
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
def verify_value(value):
    def is_int(n):
        try:
            int(n)
            return True
        except ValueError:
            return False
        else:
            return int(n).is_integer()
    
    while is_int(value)==False:
        value = input("Enter a correct value: ")
    return int(value)

In [3]:
def lista_nodos(fill, col):
    #print(np.zeros((fill, col)))
    #print()
    
    h = col-1
    th = h*(fill-1)
    
    y = fill-1
    ty = (y*col)
    lista = []
    for i in range(fill):
        for j in range(col):
            lista.append(Node(j, i))
    print("Nodos Totales:", len(lista))
    return lista

In [4]:
def total_elementos(fill, col):    
    h = col-1
    th = h*(fill-1)
    y = fill-1
    ty = (y*col)
    total = th+ty
    return total

In [5]:
class BeamLink2D11(StructElement):
    def __init__(self,nodes,E,A,I):
        StructElement.__init__(self,nodes)
        self.E = E
        self.A = A
        self.I = I

#    define node degree of freedom, left node has three dofs
#    while the right node has only two
#    def init_unknowns(self):
#        self.nodes[0].init_unknowns("Ux","Uy","Phz")
#        self.nodes[1].init_unknowns("Ux","Uy")
#        self._ndof = 2

    #transformative matrix
    def calc_T(self):
        TBase = _calc_Tbase_for_2d_beam(self.nodes)
        self._T = np.zeros((6,6))
        self._T[:3,:3] = self._T[3:,3:] = TBase

    #stiffness matrix
    def calc_ke(self):
        self._ke = _calc_ke_for_2d_beamlink(E = self.E,A = self.A,I = self.I,L = self.volume)

def _calc_ke_for_2d_beamlink(E = 1.00,A = 1.00,I =1.00,L = 1.0):
    a00 = E*A/L
    a03 = -a00
    a11 = 3.*E*I/L**3
    a12 = 3.*E*I/L**2
    a14 = -a11
    a22 = 3.*E*I/L
    T = np.array([[a00,  0.,   0.,  a03,  0.,0.],
                  [ 0., a11,  a12,  0., a14, 0.],
                  [ 0., a12,  a22,  0.,-a12, 0.],
                  [a03,  0.,   0., a00,  0., 0.],
                  [ 0., a14, -a12,  0., a11, 0.],
                  [ 0.,  0.,    0.,  0., 0., 0.]])
    return T
    
def _calc_Tbase_for_2d_beam(nodes):
    
    x1,y1 = nodes[0].x,nodes[0].y
    x2,y2 = nodes[1].x,nodes[1].y
    le = np.sqrt((x2-x1)**2+(y2-y1)**2)

    lx = (x2-x1)/le
    mx = (y2-y1)/le
    T = np.array([[lx,mx,0.],
                  [-mx,lx,0.],
                  [0.,0.,1.]])
                  
    return T

In [6]:
def lista_elementos(fill, col, E = 1, A = 1, I = 1):

    h = col-1
    th = h*(fill-1)
    
    y = fill-1
    ty = (y*col)
    lista_x = []
    cont=0
    for i in range(1, fill):
        rango = []
        for j in range(col):
            #print(j, i)
            rango.append((j, i))
        # Beam2D11((n3,n4),E,A,I)    
        nodos_l = [(Node(rango[i]), Node(rango[i+1])) for i in range(len(rango)-1)]
        elements = [Beam2D11((nodos),E,A,I) for nodos in nodos_l]
        cont+=len(elements)

        for i in elements:
            lista_x.append(i)
        
    lista_y = []
    cont2=0
    for i in range(fill):
        rango = []
        for j in range(col):
            #print(j, i)
            rango.append((i, j))
        # Beam2D11((n3,n4),E,A,I)    
        nodos_l = [(Node(rango[i]), Node(rango[i+1])) for i in range(len(rango)-1)]
        elements = [Beam2D11((nodos),E,A,I) for nodos in nodos_l]
        cont+=len(elements)

        for i in elements:
            lista_y.append(i)
        
    #print()
    #print(lista)
    print("Elementos:", cont+cont2)
    return lista_x+ lista_y

In [7]:
def coord_kxx_kxy(filas, columnas):
    rango = np.arange(columnas*3, columnas*filas*3)
    #print(rango)
    ID1 = []
    ID2 = []
    ID3 = []
    
    for i in range(0,len(rango),3):
        ID1.append(rango[i])
        
    for i in rango:
        if i not in ID1:
            ID2.append(i)
            
    for i in range(1,len(rango),3):
        ID3.append(rango[i])
        ID3.append(rango[i+1])

    kxx_coord = []
    for i in ID1:
        for j in ID1:
            kxx_coord.append([i, j])
    
    kxy_coord = []
    for i in ID1:
        for k in ID2:
            kxy_coord.append([i, k])
            
    kyy_coord = []
    for i in ID3:
        for j in ID3:
            kyy_coord.append([i, j])
    
    return kxx_coord, kxy_coord, kyy_coord

In [8]:
def k_extract(M, coord):
    L = []
    for i in coord:
        x, y = i
        L.append([x-1, y])
    valores = []
    for x, y in L:
        valor = a[x, y]
        valores.append(valor)
    return valores

In [9]:
def kxy_extract(M, coord):
    L = []
    for i in coord:
        x, y = i
        L.append([x-1, y])
    valores = []
    for x, y in L:
        valor = a[x, y]
        valores.append(valor)
    return valores

In [10]:
def kyy_extract(M, coord):
    L = []
    for i in coord:
        x, y = i
        L.append([x-1, y])
    valores = []
    for x, y in L:
        valor = a[x, y]
        valores += [valor]
    return valores

In [134]:
# materials
E = 1.0                #Modulo de elasticidad kg/cm2
A = 1.0                    #Area de secciones cm2
I = 1.0                   #Inercia de secciones cm4

Nc = 4
fx= -1000.0                                 #Carga en axial o direccion x
fy= 0.0                                    #Carga en axial o direccion y
S = (0,1,2)

In [177]:
Nniv = input("Ingrele el numero de filas de la estructura: ").strip()
Nniv = verify_value(Nniv)
Ncol = input("Ingrele el numero de columnas de la estructura: ").strip()
Ncol = verify_value(Ncol)
nodos = lista_nodos(Nniv, Ncol)
print()
print("nodos:",nodos)
print()

print("Estructura ({}x{}):".format(Nniv, Ncol))
piso = "╢═══"*(Ncol-1)+ "╢"
edificio = [piso for i in range(Nniv-1)]
floor = [j[1:-1].replace("╢", "╦") for j in edificio]
print("{}{}{}".format("╔",floor[0],"╗"))
#print(edificio)
#print(floor)
for e in range(len(edificio)-1):
    print(edificio[e])
floor = [j[1:-1].replace("╢", "╩") for j in edificio]
print("{}{}{}".format("╚",floor[0],"╝"))

Ingrele el numero de filas de la estructura: 3
Ingrele el numero de columnas de la estructura: 2
Nodos Totales: 6

nodos: [Node:(0.0, 0.0), Node:(1.0, 0.0), Node:(0.0, 1.0), Node:(1.0, 1.0), Node:(0.0, 2.0), Node:(1.0, 2.0)]

Estructura (3x2):
╔═══╗
╢═══╢
╚═══╝


In [136]:
### %%
# Generacion de nodos
n = []
elementos = lista_elementos(Nniv,Ncol)
L = [0,1,2,3]
H = [0,1,2,3]
for i in range(0,Nniv):
    for j in range(0,Ncol):
        nnod= j+Ncol*i
        x =L[j]
        y =H[i]
        n.append(Node(x,y))

n

Elementos: 10


[Node:(0.0, 0.0),
 Node:(1.0, 0.0),
 Node:(2.0, 0.0),
 Node:(0.0, 1.0),
 Node:(1.0, 1.0),
 Node:(2.0, 1.0),
 Node:(0.0, 2.0),
 Node:(1.0, 2.0),
 Node:(2.0, 2.0)]

In [178]:
def f(x):
    return x**2

In [180]:
z=3
print(f(z))

9


In [137]:
#%%
###    #Creacion de elementos    
v = []
for j in range(1,Nniv):
    for i in range(Ncol-1):
        v.append(Beam2D11((n[j*Ncol+i],n[j*Ncol+i+1]),E,A,I))  
v

[Beam2D11 Element: (Node:(0.0, 1.0), Node:(1.0, 1.0)),
 Beam2D11 Element: (Node:(1.0, 1.0), Node:(2.0, 1.0)),
 Beam2D11 Element: (Node:(0.0, 2.0), Node:(1.0, 2.0)),
 Beam2D11 Element: (Node:(1.0, 2.0), Node:(2.0, 2.0))]

In [138]:
c = []
for i in range(1,Nniv):
    for j in range(Ncol):
        #print(j)
        c.append(Beam2D11((n[(i-1)*Ncol+j],n[Ncol*(i-1)+j+Ncol]),E,A,I))
c

[Beam2D11 Element: (Node:(0.0, 0.0), Node:(0.0, 1.0)),
 Beam2D11 Element: (Node:(1.0, 0.0), Node:(1.0, 1.0)),
 Beam2D11 Element: (Node:(2.0, 0.0), Node:(2.0, 1.0)),
 Beam2D11 Element: (Node:(0.0, 1.0), Node:(0.0, 2.0)),
 Beam2D11 Element: (Node:(1.0, 1.0), Node:(1.0, 2.0)),
 Beam2D11 Element: (Node:(2.0, 1.0), Node:(2.0, 2.0))]

In [174]:
s = System()
s.add_nodes(n)
s.add_elements(v,c)
s.add_node_force(Nc,Fx = fx,Fy = fy)
a = s.KG
s.add_fixed_sup(S)
s.solve
plt.show()

In [175]:
s

2D System: 
Nodes: 9
Elements: 10

In [153]:
kxx_coord, kxy_coord, kyy_coord = coord_kxx_kxy(Nniv, Ncol)
#print(kxx_coord)
#print(kxy_coord)
#print(kyy_coord)

In [154]:
a.shape

(27, 27)

In [166]:
kxx = kxx_extract(a, kxx_coord)
kxx = np.array(kxx)
#kxx

In [167]:
kxx = kxx.reshape((6, 6))
#kxx

In [168]:
kxy = kxy_extract(a, kxy_coord)
kxy = np.array(kxy)
#kxy

In [169]:
kxy = kxy.reshape((6, 12))
#kxy

In [170]:
kyx = kxy.T
#kyx

In [171]:
kyy = kyy_extract(a, kyy_coord)
kyy = np.array(kyy)
#kyy

In [172]:
kyy = kyy.reshape((12, 12))
#kyy

In [173]:
kxx - np.dot(kxy, np.dot(np.linalg.inv(kyy), kyx))

array([[ 0.00000000e+00,  0.00000000e+00,  8.00000000e+00,
         2.00000000e+00,  0.00000000e+00,  2.00000000e+00],
       [ 2.72932109e-01, -1.10648237e+01, -5.01209964e+00,
         8.55948566e+00, -1.06008376e+01, -8.79715302e+00],
       [ 8.61878453e-01,  4.37569061e+00,  0.00000000e+00,
        -4.37569061e+00,  1.62872928e+01, -1.77635684e-15],
       [ 5.88946344e-01, -2.55948566e+00,  5.01209964e+00,
         1.10648237e+01, -1.11186961e+00,  1.47971530e+01],
       [-5.97112129e+00,  3.02841077e+00,  3.54021352e+00,
         5.76874226e+00, -4.27934174e+00, -2.91814947e-01],
       [ 6.62983425e-02, -1.27403315e+01,  0.00000000e+00,
         6.74033149e+00,  4.02209945e+00, -1.77635684e-14]])

In [165]:
for i in n:
    print(i.disp)

{'Ux': 0.0, 'Uy': 0.0, 'Phz': 0.0}
{'Ux': 0.0, 'Uy': 0.0, 'Phz': 0.0}
{'Ux': 0.0, 'Uy': 0.0, 'Phz': 0.0}
{'Ux': None, 'Uy': None, 'Phz': None}
{'Ux': None, 'Uy': None, 'Phz': None}
{'Ux': None, 'Uy': None, 'Phz': None}
{'Ux': None, 'Uy': None, 'Phz': None}
{'Ux': None, 'Uy': None, 'Phz': None}
{'Ux': None, 'Uy': None, 'Phz': None}
